In [1]:
import pandas as pd
import numpy as np

np.random.seed(0)

# Load the store sales dataset
excel_file = pd.ExcelFile('data/store_sales.xlsx')
sheet_names = excel_file.sheet_names
features = ['F', 'D', 'Unit.Price']
target = 'Total.Volume'

sheet_names, len(sheet_names), features, target

(['StoreWeekSalescarbbev_modify',
  'StoreWeekSalescigets',
  'StoreWeekSalescoffee',
  'StoreWeekSalescoldcer',
  'StoreWeekSalesdeod',
  'StoreWeekSalesdiapers',
  'StoreWeekSalesfactiss',
  'StoreWeekSalesfzdinent',
  'StoreWeekSalesfzpizza',
  'StoreWeekSaleshotdog',
  'StoreWeekSaleslaundet',
  'StoreWeekSalesmargbutr',
  'StoreWeekSalesmayo',
  'StoreWeekSalesmustketc',
  'StoreWeekSalespaptowl',
  'StoreWeekSalespeanbutr',
  'StoreWeekSalesshamp',
  'StoreWeekSalessoup',
  'StoreWeekSalesspagsauc',
  'StoreWeekSalessugarsub',
  'StoreWeekSalestoitisu',
  'StoreWeekSalestoothpa',
  'StoreWeekSalesyogurt',
  'StoreWeekSalesbeer_modify'],
 24,
 ['F', 'D', 'Unit.Price'],
 'Total.Volume')

In [2]:
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import time


def evaluate_model(model, X, y, n=1, aggregate=True):
    data = []

    for random_state in range(n):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state)

        start_time = time.time()
        model.fit(X_train, y_train)
        end_time = time.time()
        train_time = end_time - start_time

        start_time = time.time()
        predictions = model.predict(X_test)
        end_time = time.time()
        comp_time = end_time - start_time

        r2 = r2_score(y_test, predictions)
        mae = mean_absolute_error(y_test, predictions)
        mse = mean_squared_error(y_test, predictions)
        n_iter = model.n_iter_ if hasattr(model, 'n_iter_') else 0

        data.append([r2, mae, mse, train_time, comp_time, n_iter])

    df = pd.DataFrame(
        data, columns=['r2', 'mae', 'mse', 'train_time', 'comp_time', 'n_iter'])

    return df.mean() if aggregate else df

In [3]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV


def best_estimator(model, param_grid, X, y):
    search = GridSearchCV(model, param_grid)
    search.fit(X, y)
    return search.best_estimator_

In [4]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge, ARDRegression, SGDRegressor, PassiveAggressiveRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from models_sklearn import Ensemble, MLP, FONN1, FONN2, TREENN1, TREENN2


def test_models(X, y):
    # Initialize standard models
    models = {
        # 'Linear Regression': LinearRegression(),
        # 'Ridge Regression': Ridge(),
        # 'Lasso Regression': Lasso(random_state=42),
        # 'ElasticNet Regression': ElasticNet(random_state=42),
        # 'Bayesian Ridge Regression': BayesianRidge(),
        # 'ARD Regression': ARDRegression(),
        # 'SGD Regressor': SGDRegressor(random_state=42),
        # 'Passive Aggressive Regressor': PassiveAggressiveRegressor(random_state=42),
        # 'Support Vector Regression': SVR(),
        # 'MLP Regressor': MLPRegressor(hidden_layer_sizes=(40,), max_iter=2000, learning_rate_init=1e-2, random_state=42, early_stopping=False),
        # 'Random Forest Regressor': RandomForestRegressor(n_estimators=100, random_state=42),
        # 'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
        # 'XGBoost Regressor': XGBRegressor(random_state=42),
        # 'AdaBoost Regressor': AdaBoostRegressor(random_state=42),
        # 'Bagging Regressor': BaggingRegressor(random_state=42),
        # 'ExtraTrees Regressor': ExtraTreesRegressor(random_state=42),
        # 'HistGradientBoosting Regressor': HistGradientBoostingRegressor(random_state=42),
        # 'Stacking Regressor': StackingRegressor(estimators=[
        #     ('lr', LinearRegression()),
        #     ('rf', RandomForestRegressor(n_estimators=10, random_state=42))
        # ], final_estimator=Ridge(random_state=42)),
        # 'Voting Regressor': VotingRegressor(estimators=[
        #     ('lr', LinearRegression()),
        #     ('rf', RandomForestRegressor(n_estimators=10, random_state=42)),
        #     ('gb', GradientBoostingRegressor(random_state=42))
        # ])
    }

    ensemble_param_grid = {
    }

    mlp_param_grid = {
        'max_iter': [2000],
        'learning_rate': ['constant'],
        'learning_rate_init': [1e-2],
        'tol': [1e-4],
        'early_stopping': [True]
    }

    models['Tree'] = best_estimator(Ensemble(1), ensemble_param_grid, X, y) 
    models['Ensemble 5'] = best_estimator(Ensemble(5), ensemble_param_grid, X, y)
    models['MLP 5'] = best_estimator(MLP(5), mlp_param_grid, X, y)
    models['FONN1 5 5'] = best_estimator(FONN1(5, (10,)), mlp_param_grid, X, y)
    models['FONN2 5 5'] = best_estimator(FONN2(5, (10,)), mlp_param_grid, X, y)
    models['TREENN1 5'] = best_estimator(TREENN1((6,)), mlp_param_grid, X, y)
    models['TREENN2 5'] = best_estimator(TREENN2((6,)), mlp_param_grid, X, y)
    models['Ensemble 10'] = best_estimator(Ensemble(10), ensemble_param_grid, X, y)
    models['MLP 10'] = best_estimator(MLP(10), mlp_param_grid, X, y)
    models['FONN1 5 10'] = best_estimator(FONN1(5, (15,)), mlp_param_grid, X, y)
    models['FONN2 5 10'] = best_estimator(FONN2(5, (15,)), mlp_param_grid, X, y)
    models['TREENN1 10'] = best_estimator(TREENN1((11,)), mlp_param_grid, X, y)
    models['TREENN2 10'] = best_estimator(TREENN2((11,)), mlp_param_grid, X, y)
    models['Ensemble 40'] = best_estimator(Ensemble(40), ensemble_param_grid, X, y)
    models['MLP 40'] = best_estimator(MLP(40), mlp_param_grid, X, y)
    models['FONN1 5 40'] = best_estimator(FONN1(5, (45,)), mlp_param_grid, X, y)
    models['FONN2 5 40'] = best_estimator(FONN2(5, (45,)), mlp_param_grid, X, y)
    models['TREENN1 40'] = best_estimator(TREENN1((41,)), mlp_param_grid, X, y)
    models['TREENN2 40'] = best_estimator(TREENN2((41,)), mlp_param_grid, X, y)
    models['Ensemble 100'] = best_estimator(Ensemble(100), ensemble_param_grid, X, y)
    models['MLP 100'] = best_estimator(MLP(100), mlp_param_grid, X, y)
    models['FONN1 5 100'] = best_estimator(FONN1(5, (105,)), mlp_param_grid, X, y)
    models['FONN2 5 100'] = best_estimator(FONN2(5, (105,)), mlp_param_grid, X, y)
    models['TREENN1 100'] = best_estimator(TREENN1((101,)), mlp_param_grid, X, y)
    models['TREENN2 100'] = best_estimator(TREENN2((101,)), mlp_param_grid, X, y)

    # Train and evaluate models
    results = []
    for name, model in models.items():
        result = evaluate_model(model, X, y, n=3).values
        results.append([name, *result])

    results_df = pd.DataFrame(
        results, columns=['model', 'r2', 'mae', 'mse', 'train_time', 'comp_time', 'n_iter'])
    results_df.set_index('model', inplace=True)

    return results_df

In [5]:
from sklearn.preprocessing import StandardScaler


def test_sheet(sheet_name):
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    iri_key_counts = df['IRI_KEY'].value_counts()
    iri_keys = list(map(int, iri_key_counts[iri_key_counts > 300].index))
    print(sheet_name, len(iri_keys))

    sheet_data = []

    for iri_key in iri_keys:
        iri_key = int(iri_key)
        df_iri = df[df['IRI_KEY'] == iri_key]
        X = df_iri[features]
        y = df_iri[target].values

        scaler_X = StandardScaler()
        X = scaler_X.fit_transform(X)
        scaler_y = StandardScaler()
        y = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()

        results = test_models(X, y)
        # results['sheet_name'] = sheet_name
        # results['iri_key'] = iri_key
        # sheet_data.append(results)
        results[f'mse_{sheet_name}_{iri_key}'] = results['mse']
        sheet_data.append(results[f'mse_{sheet_name}_{iri_key}'])

    sheet_df = pd.DataFrame(sheet_data)

    return sheet_df

In [6]:
from sklearn.utils import shuffle

sheet_names = shuffle(sheet_names, random_state=0, n_samples=2)

df = []

for sheet_name in sheet_names:  # type: ignore
    results = test_sheet(sheet_name)
    df.append(results)

StoreWeekSalesmargbutr 16
StoreWeekSaleslaundet 16


,mse_StoreWeekSalesmargbutr_6001821,mse_StoreWeekSalesmargbutr_648368,mse_StoreWeekSalesmargbutr_279300,mse_StoreWeekSalesmargbutr_400003,mse_StoreWeekSalesmargbutr_270862,mse_StoreWeekSalesmargbutr_231720,mse_StoreWeekSalesmargbutr_252570,mse_StoreWeekSalesmargbutr_659827,mse_StoreWeekSalesmargbutr_241565,mse_StoreWeekSalesmargbutr_237277,...,mse_StoreWeekSaleslaundet_252570,mse_StoreWeekSaleslaundet_659827,mse_StoreWeekSaleslaundet_241565,mse_StoreWeekSaleslaundet_237277,mse_StoreWeekSaleslaundet_291276,mse_StoreWeekSaleslaundet_273920,mse_StoreWeekSaleslaundet_232633,mse_StoreWeekSaleslaundet_233246,mse_StoreWeekSaleslaundet_532639,mse_StoreWeekSaleslaundet_533864
model,,,,,,,,,,,,,,,,,,,,,
Tree,0.545237,0.554553,0.812628,0.658576,0.774760,0.404478,0.413752,0.503187,0.415552,0.625687,...,0.635972,0.806726,1.567514,1.121980,1.862134,1.477885,0.567014,1.643040,1.593015,1.362446
Ensemble 5,0.345805,0.352997,0.499783,0.377195,0.397725,0.168323,0.314323,0.281855,0.315664,0.321594,...,0.568312,0.477182,1.116746,0.774602,0.917050,1.095101,0.630570,1.215221,1.135881,0.685742
MLP 5,0.344005,0.349022,0.409179,0.331875,0.548175,0.404569,0.288907,0.252268,0.234403,0.392974,...,0.548763,0.431371,0.971535,0.857163,0.846269,0.857147,0.529915,1.014230,0.784672,0.601431
FONN1 5 5,0.387132,0.447352,0.801176,0.365249,0.495662,0.238392,0.354431,0.379261,0.325530,0.391929,...,0.567378,0.437262,1.148864,0.774772,1.284211,1.391234,0.583582,1.260607,1.256973,0.726780
FONN2 5 5,0.366499,2.427106,0.551924,0.351349,0.334379,0.366429,0.434587,0.310108,0.274331,0.477182,...,0.565325,0.430158,0.848050,0.732310,0.829882,0.889558,0.427422,0.935536,0.823166,0.786201
TREENN1 5,0.494860,1.821909,0.726603,0.396129,0.618772,0.308701,0.406835,0.267841,0.315371,0.424713,...,0.569589,0.473682,1.135647,0.745950,0.929143,1.013216,0.479580,1.144298,1.019082,0.644666
TREENN2 5,0.361363,0.541910,0.406345,0.337253,0.403180,0.598713,0.526678,0.257946,0.185244,0.427835,...,0.566289,0.504450,0.947281,0.732105,0.768702,0.936647,0.526836,0.959886,0.911955,0.626897
Ensemble 10,0.383420,0.373251,0.457310,0.335313,0.413693,0.178745,0.232441,0.293458,0.266823,0.291658,...,0.488457,0.390852,0.995041,0.664266,0.926789,1.035129,0.485993,1.233437,0.924411,0.691559
MLP 10,0.340685,0.403147,0.466141,0.330540,0.270553,0.306507,0.322811,0.307500,0.218941,0.327243,...,0.491978,0.403719,0.798305,0.848821,0.688355,0.838638,0.657688,1.143213,0.742856,0.576388


In [10]:
dataset_mse = pd.concat(df).T
dataset_rank = dataset_mse.rank(numeric_only=True)
dataset_rank = dataset_rank.assign(Mean=dataset_rank.mean(1), Stddev=dataset_rank.std(1))
dataset_rank.to_csv("output/dataset_rank.csv")

dataset_mse = dataset_mse.assign(Mean=dataset_mse.mean(1), Stddev=dataset_mse.std(1))
dataset_mse.to_csv("output/dataset_mse.csv")

In [11]:
dataset_mse

,mse_StoreWeekSalesmargbutr_6001821,mse_StoreWeekSalesmargbutr_648368,mse_StoreWeekSalesmargbutr_279300,mse_StoreWeekSalesmargbutr_400003,mse_StoreWeekSalesmargbutr_270862,mse_StoreWeekSalesmargbutr_231720,mse_StoreWeekSalesmargbutr_252570,mse_StoreWeekSalesmargbutr_659827,mse_StoreWeekSalesmargbutr_241565,mse_StoreWeekSalesmargbutr_237277,...,mse_StoreWeekSaleslaundet_241565,mse_StoreWeekSaleslaundet_237277,mse_StoreWeekSaleslaundet_291276,mse_StoreWeekSaleslaundet_273920,mse_StoreWeekSaleslaundet_232633,mse_StoreWeekSaleslaundet_233246,mse_StoreWeekSaleslaundet_532639,mse_StoreWeekSaleslaundet_533864,Mean,Stddev
model,,,,,,,,,,,,,,,,,,,,,
Tree,0.545237,0.554553,0.812628,0.658576,0.774760,0.404478,0.413752,0.503187,0.415552,0.625687,...,1.567514,1.121980,1.862134,1.477885,0.567014,1.643040,1.593015,1.362446,0.875069,0.439896
Ensemble 5,0.345805,0.352997,0.499783,0.377195,0.397725,0.168323,0.314323,0.281855,0.315664,0.321594,...,1.116746,0.774602,0.917050,1.095101,0.630570,1.215221,1.135881,0.685742,0.570762,0.312696
MLP 5,0.344005,0.349022,0.409179,0.331875,0.548175,0.404569,0.288907,0.252268,0.234403,0.392974,...,0.971535,0.857163,0.846269,0.857147,0.529915,1.014230,0.784672,0.601431,0.538786,0.266172
FONN1 5 5,0.387132,0.447352,0.801176,0.365249,0.495662,0.238392,0.354431,0.379261,0.325530,0.391929,...,1.148864,0.774772,1.284211,1.391234,0.583582,1.260607,1.256973,0.726780,0.647615,0.365584
FONN2 5 5,0.366499,2.427106,0.551924,0.351349,0.334379,0.366429,0.434587,0.310108,0.274331,0.477182,...,0.848050,0.732310,0.829882,0.889558,0.427422,0.935536,0.823166,0.786201,0.611012,0.418295
TREENN1 5,0.494860,1.821909,0.726603,0.396129,0.618772,0.308701,0.406835,0.267841,0.315371,0.424713,...,1.135647,0.745950,0.929143,1.013216,0.479580,1.144298,1.019082,0.644666,0.650091,0.346485
TREENN2 5,0.361363,0.541910,0.406345,0.337253,0.403180,0.598713,0.526678,0.257946,0.185244,0.427835,...,0.947281,0.732105,0.768702,0.936647,0.526836,0.959886,0.911955,0.626897,0.553481,0.243091
Ensemble 10,0.383420,0.373251,0.457310,0.335313,0.413693,0.178745,0.232441,0.293458,0.266823,0.291658,...,0.995041,0.664266,0.926789,1.035129,0.485993,1.233437,0.924411,0.691559,0.532626,0.291078
MLP 10,0.340685,0.403147,0.466141,0.330540,0.270553,0.306507,0.322811,0.307500,0.218941,0.327243,...,0.798305,0.848821,0.688355,0.838638,0.657688,1.143213,0.742856,0.576388,0.507906,0.248215


In [12]:
dataset_rank

,mse_StoreWeekSalesmargbutr_6001821,mse_StoreWeekSalesmargbutr_648368,mse_StoreWeekSalesmargbutr_279300,mse_StoreWeekSalesmargbutr_400003,mse_StoreWeekSalesmargbutr_270862,mse_StoreWeekSalesmargbutr_231720,mse_StoreWeekSalesmargbutr_252570,mse_StoreWeekSalesmargbutr_659827,mse_StoreWeekSalesmargbutr_241565,mse_StoreWeekSalesmargbutr_237277,...,mse_StoreWeekSaleslaundet_241565,mse_StoreWeekSaleslaundet_237277,mse_StoreWeekSaleslaundet_291276,mse_StoreWeekSaleslaundet_273920,mse_StoreWeekSaleslaundet_232633,mse_StoreWeekSaleslaundet_233246,mse_StoreWeekSaleslaundet_532639,mse_StoreWeekSaleslaundet_533864,Mean,Stddev
model,,,,,,,,,,,,,,,,,,,,,
Tree,25.0,20.0,25.0,25.0,25.0,23.0,21.0,25.0,25.0,25.0,...,25.0,25.0,25.0,25.0,20.0,25.0,25.0,25.0,24.34375,1.536846
Ensemble 5,8.0,5.0,15.0,18.0,13.0,2.0,11.0,15.0,21.0,4.0,...,17.0,17.0,15.0,18.0,23.0,17.0,21.0,15.0,14.06250,5.423589
MLP 5,7.0,4.0,3.0,6.0,21.0,24.0,9.0,6.0,9.0,18.0,...,14.0,21.0,12.0,6.0,14.0,9.0,9.0,9.0,11.31250,6.331883
FONN1 5 5,18.0,16.0,24.0,17.0,19.0,8.0,17.0,24.0,22.0,17.0,...,19.0,18.0,23.0,24.0,22.0,20.0,24.0,19.0,19.00000,4.212118
FONN2 5 5,13.0,25.0,20.0,13.0,9.0,20.0,22.0,21.0,15.0,24.0,...,9.0,11.0,11.0,9.0,2.0,1.0,11.0,22.0,14.18750,6.321686
TREENN1 5,24.0,24.0,23.0,20.0,24.0,15.0,20.0,14.0,20.0,19.0,...,18.0,13.0,17.0,16.0,9.0,14.0,18.0,13.0,18.12500,4.163978
TREENN2 5,10.0,19.0,1.0,8.0,14.0,25.0,25.0,9.0,3.0,20.0,...,13.0,10.0,6.0,11.0,13.0,2.0,13.0,11.0,12.56250,7.264307
Ensemble 10,17.0,11.0,10.0,7.0,15.0,4.0,3.0,16.0,14.0,1.0,...,15.0,1.0,16.0,17.0,11.0,18.0,14.0,16.0,11.18750,5.479802
MLP 10,6.0,14.0,11.0,5.0,2.0,14.0,12.0,19.0,8.0,5.0,...,5.0,20.0,3.0,5.0,24.0,13.0,6.0,6.0,9.71875,6.091953


In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd

# # Convert cv_results_ to a DataFrame
# results = pd.DataFrame(search.cv_results_)

# # Pivot the DataFrame to create a matrix for the heatmap
# heatmap_data = results.sort_values('iter').pivot_table(
#             index='param_learning_rate',
#             columns='param_epochs',
#             values='mean_test_score',
#             aggfunc='last',
#         )

# # Plot the heatmap
# plt.figure(figsize=(10, 6))
# sns.heatmap(heatmap_data, annot=True, cmap='viridis')
# plt.title('HalvingGridSearchCV Results')
# plt.xlabel('Number of Estimators')
# plt.ylabel('Learning Rate')
# plt.show()


In [ ]:
results

model,Tree,Ensemble 5,MLP 5,FONN1 5 5,FONN2 5 5,TREENN1 5,TREENN2 5,Ensemble 10,MLP 10,FONN1 5 10,...,FONN1 5 40,FONN2 5 40,TREENN1 40,TREENN2 40,Ensemble 100,MLP 100,FONN1 5 100,FONN2 5 100,TREENN1 100,TREENN2 100
mse_StoreWeekSaleslaundet_6001821,1.424227,0.994571,1.089854,1.214628,1.266833,1.113499,0.976719,0.974904,0.931069,1.327239,...,1.056657,1.005232,1.146936,0.901965,0.921325,0.918250,1.078996,0.924735,1.099154,0.882836
mse_StoreWeekSaleslaundet_648368,0.975666,0.640998,0.646140,0.659219,0.553012,0.695548,0.604898,0.633491,0.522481,0.769917,...,0.811608,0.493145,0.700901,0.581733,0.629983,0.510715,0.711763,0.569494,0.842618,0.511343
mse_StoreWeekSaleslaundet_279300,0.797843,0.672743,0.402126,0.744073,0.474086,0.586475,0.393394,0.581896,0.399665,0.669393,...,0.606851,0.381443,0.568338,0.415188,0.548296,0.426895,0.726801,0.460213,0.539960,0.404531
mse_StoreWeekSaleslaundet_400003,1.601449,1.138716,1.004696,1.362395,1.053221,1.099379,1.029715,1.046704,1.043198,1.313953,...,1.283421,1.017949,1.179116,1.059630,1.029019,1.042331,1.287893,0.985966,1.304348,1.039206
mse_StoreWeekSaleslaundet_270862,0.660127,0.392545,0.480927,0.450569,0.407633,0.410884,0.573953,0.334719,0.435121,0.366420,...,0.445099,0.394713,0.487052,0.420961,0.333563,0.424368,0.403679,0.457890,0.428432,0.429617
mse_StoreWeekSaleslaundet_231720,0.751145,0.409161,0.830631,0.392606,0.576025,0.485906,0.627572,0.434271,0.484582,0.337253,...,0.329613,0.507732,0.292410,0.523133,0.354615,0.553624,0.264835,0.479040,0.651625,0.482137
mse_StoreWeekSaleslaundet_252570,0.635972,0.568312,0.548763,0.567378,0.565325,0.569589,0.566289,0.488457,0.491978,0.653438,...,0.572500,0.490243,0.534503,0.601995,0.504988,0.517625,0.470942,0.530085,0.650528,0.509698
mse_StoreWeekSaleslaundet_659827,0.806726,0.477182,0.431371,0.437262,0.430158,0.473682,0.504450,0.390852,0.403719,0.415725,...,0.479772,0.377743,0.468132,0.425604,0.376409,0.371302,0.492391,0.351042,0.456071,0.343139
mse_StoreWeekSaleslaundet_241565,1.567514,1.116746,0.971535,1.148864,0.848050,1.135647,0.947281,0.995041,0.798305,1.299932,...,1.312754,0.834511,1.221060,0.845943,0.934764,0.780322,1.177955,0.731739,1.207521,0.778999
mse_StoreWeekSaleslaundet_237277,1.121980,0.774602,0.857163,0.774772,0.732310,0.745950,0.732105,0.664266,0.848821,0.889630,...,0.731121,0.668969,0.757930,0.774362,0.677446,0.683806,0.881916,0.729175,0.939776,0.743237
